In [0]:
import translator_helper as helper
import os
import preprocess_util as putil
source_path = "./raw_data/parallel_sentence_corpus/amh.txt"
target_path = "./raw_data/parallel_sentence_corpus/amh.txt"
source_vocab_mapping = "./amh_vocab_mapping.p"
target_vocab_mapping = "./amh_vocab_mapping.p"
PREPROCESS_SAVE_PATH = "preprocssed_data.p"



>>>>> This is L3Morpho, version 3.0 <<<<<
>>>>>  and HornMorpho, version 2.5  <<<<<


In [0]:
src_int_to_vocab, src_vocab_to_int = putil.load_file(source_vocab_mapping)
tgt_int_to_vocab, tgt_vocab_to_int = putil.load_file(target_vocab_mapping)

In [0]:
helper.preprocess_and_save(source_path, target_path,
                           src_int_to_vocab, src_vocab_to_int,
                           tgt_int_to_vocab, tgt_vocab_to_int,
                           PREPROCESS_SAVE_PATH)
print("processed data saved...")

Loading morphological data for Amharic ...



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [0]:
#checkpoint processed data saved

39770

In [0]:
import translator_helper as helper
import tensorflow as tf
import numpy as np
PREPROCESS_SAVE_PATH = "preprocssed_data.p"
(src_int_text, tgt_int_text), (src_int_to_vocab, tgt_int_to_vocab), (src_vocab_to_int, tgt_vocab_to_int) = helper.load_preprocessed_data(PREPROCESS_SAVE_PATH)


In [0]:
!ln -sf ./drive/My\ Drive/Amh2Amh_v3_checked/HornMorph/ ./

In [0]:
from tensorflow.python.layers.core import Dense

print(tf.__version__)

if not tf.test.gpu_device_name():
    print('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

1.14.0
No GPU found. Please use a GPU to train your neural network.


In [0]:
# prepare_model_input
def model_inputs():
    src_input = tf.placeholder(tf.int32, (None, None), name="src_input")
    target = tf.placeholder(tf.int32, (None, None), name="targets")
    learning_rate = tf.placeholder(tf.float32, [], name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, [], name="keep_prob")
    src_seq_len = tf.placeholder(tf.int32, (None, ), name="src_seq_len")
    tgt_seq_len = tf.placeholder(tf.int32, (None, ), name="tgt_seq_len")
    max_tgt_seq = tf.reduce_max(tgt_seq_len)
    
    return src_input, target, learning_rate, keep_prob, src_seq_len, tgt_seq_len, max_tgt_seq

In [0]:
def prepare_decoder_input(target, target_to_int, batch_size):
    sliced = tf.strided_slice(target, [0,0], [batch_size, -1], [1,1])
    decoder_input = tf.concat([tf.fill([batch_size,1], target_to_int['<GO>']), sliced], 1)
    return decoder_input
    

In [0]:
def encoder(enc_inputs, src_seq_len, enc_embedding_size, src_vocab_size, rnn_size, num_layers, keep_prob=0.5):
    embed = tf.contrib.layers.embed_sequence(enc_inputs, src_vocab_size, enc_embedding_size)
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    # returns output and state
    outputs, state = tf.nn.dynamic_rnn(cell, embed, sequence_length=src_seq_len, dtype=tf.float32)
    return outputs, state   

In [0]:
def decoder_train(dec_embeded_input, enc_state, tgt_seq_len, max_tgt_len, dec_cell, keep_prob, output_layer):
    
    train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embeded_input, 
                                                     sequence_length=tgt_seq_len, 
                                                     time_major=False)
    dec_train = tf.contrib.seq2seq.BasicDecoder(dec_cell, train_helper, enc_state, output_layer)
    
    decoder_train_output = tf.contrib.seq2seq.dynamic_decode(dec_train,
                                                       impute_finished=True,
                                                       maximum_iterations=max_tgt_len)[0]
    return decoder_train_output
    

In [0]:
def decoder_inference(dec_embedding_mat, enc_state, max_tgt_len, start_id, end_id, batch_size, dec_cell, keep_prob, output_layer):
    
    start_tokens = tf.tile(tf.constant([start_id], dtype=tf.int32), [batch_size], name='start_tokens')
    
    infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embedding_mat, start_tokens, end_id)
    
    dec_inference = tf.contrib.seq2seq.BasicDecoder(dec_cell, infer_helper, enc_state, output_layer)
    
    decoder_infer_output = tf.contrib.seq2seq.dynamic_decode(dec_inference,
                                                       impute_finished=True,
                                                       maximum_iterations=max_tgt_len)[0]
    return decoder_infer_output
    

In [0]:
def decoder(dec_input, enc_state, 
            tgt_seq_len, max_tgt_len, 
            tgt_vocab_to_int, tgt_vocab_size, 
            rnn_size, num_layers, dec_embed_size,
            batch_size, keep_prob):
    embed_mat = tf.Variable(tf.random_uniform([tgt_vocab_size, dec_embed_size]))
    dec_embeded_input = tf.nn.embedding_lookup(embed_mat, dec_input)
    
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    output_layer = Dense(tgt_vocab_size, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode"):
        dec_train_output = decoder_train(dec_embeded_input, enc_state,
                                         tgt_seq_len, max_tgt_len,
                                         dec_cell, keep_prob, output_layer)
    start_id = tgt_vocab_to_int["<GO>"]
    end_id = tgt_vocab_to_int["<EOS>"]
   
    with tf.variable_scope("decode", reuse=True):
        dec_infer_output = decoder_inference(embed_mat, enc_state, max_tgt_len,
                                                start_id, end_id, batch_size,
                                                dec_cell, keep_prob, output_layer)
    return dec_train_output, dec_infer_output
    

In [0]:
def EncoderDecoder(inputs, targets,
                   enc_embedding_size, dec_embedding_size,
                   src_seq_len, tgt_seq_len, max_tgt_len, 
                   tgt_vocab_size, src_vocab_size, tgt_vocab_to_int,
                   rnn_size, num_layers, batch_size, keep_prob):
    
    outputs, state = encoder(inputs, src_seq_len, enc_embedding_size,
                      src_vocab_size, rnn_size, num_layers, keep_prob)
    
    dec_procssed_input = prepare_decoder_input(targets, tgt_vocab_to_int, batch_size)
    
    decoder_train_output, dec_infer_output = decoder(dec_procssed_input, state, tgt_seq_len, max_tgt_len,
                                                      tgt_vocab_to_int, tgt_vocab_size, rnn_size, num_layers,
                                                      dec_embedding_size, batch_size, keep_prob)
    return decoder_train_output, dec_infer_output

In [0]:
def pad_sentences(batch, pad_id):
    
    seq_len = [len(sen) for sen in batch]
    #print(seq_len)
    max_len = max(seq_len)
        
    return [sent + [pad_id] * (max_len - len(sent)) for sent in batch]
    

In [0]:
def getBatches(src_int_text, tgt_int_text, batch_size, src_pad_id, tgt_pad_id):
    assert len(src_int_text) == len(tgt_int_text), "source sentences != target sentences"
    n_batches = len(src_int_text) // batch_size
    
    total_sen_in_batch = batch_size * n_batches
    src_int_sents = src_int_text[:total_sen_in_batch + 1]
    tgt_int_sents = tgt_int_text[:total_sen_in_batch + 1]
    batches = []
    for idx in range(0, len(src_int_sents), batch_size):
        src_batch = src_int_sents[idx:idx + batch_size]
        tgt_batch = tgt_int_sents[idx:idx + batch_size]
        src_padded_batch = pad_sentences(src_batch, src_pad_id)
        tgt_padded_batch = pad_sentences(tgt_batch, tgt_pad_id)
        batches.append((src_padded_batch, tgt_padded_batch))
    return batches
        

In [0]:
train_graph = tf.Graph()
(src_int_text, tgt_int_text), (src_vocab_to_int, tgt_vocab_to_int), (src_int_to_vocab, tgt_int_to_vocab)  = helper.load_preprocessed_data(PREPROCESS_SAVE_PATH)
max_tgt_len = max([len(sen) for sen in src_int_text])

In [0]:
epochs = 10
batch_size = 256
lr = 0.001
keep_probe = 0.75
rnn_size = 512
num_layers = 2
enc_embedding_size = 256
dec_embedding_size = 256
display_step = 10

In [0]:

with train_graph.as_default():
    src_inputs, targets, learning_rate, keep_prob, src_seq_len, tgt_seq_len, max_tgt_seq = model_inputs()
    train_logits, infer_logits = EncoderDecoder(src_inputs, targets,
                                                enc_embedding_size, dec_embedding_size,
                                                src_seq_len, tgt_seq_len, max_tgt_len, 
                                                len(src_vocab_to_int), len(tgt_vocab_to_int), tgt_vocab_to_int,
                                                rnn_size, num_layers, batch_size, keep_prob)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(infer_logits.sample_id, name='predictions')
    masks = tf.sequence_mask(tgt_seq_len, max_tgt_seq, dtype=tf.float32, name='masks')
    
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_optimzer = optimizer.apply_gradients(capped_gradients)
    

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where

In [0]:
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    src_pad_id = src_vocab_to_int['<PAD>']
    tgt_pad_id = tgt_vocab_to_int['<PAD>']
    batches = getBatches(src_int_text, tgt_int_text, batch_size, src_pad_id, tgt_pad_id)
    for epoch_i in range(epochs):
        for batch_ind, (src_batch, tgt_batch) in enumerate(batches):
            srce_seq_length = [len(sent) for sent in src_batch]
            tgte_seq_length = [len(sent) for sent in tgt_batch]
            _, loss = sess.run(
                [train_optimzer, cost],
                {src_inputs: src_batch,
                 targets: tgt_batch,
                 learning_rate: lr,
                 tgt_seq_len: tgte_seq_length,
                 src_seq_len: srce_seq_length,
                 keep_prob: keep_probe
                })


            if batch_ind % display_step == 0 and batch_ind > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {src_inputs: src_batch,
                     src_seq_len: srce_seq_length,
                     tgt_seq_len: tgte_seq_length,
                     keep_prob: 1.0})

                print('Epoch {:>3} Batch {:>4}/{} - Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

KeyboardInterrupt: 